In [9]:
import pandas as pd
from pathlib import Path
import argparse
from handling_acc_files import HelperMethod
from create_reference_from_tsv_and_pepxml import ReferenceWriter
from collections import defaultdict
from create_PSM_df import PSM_FDR
import pickle
from pathlib import Path
import argparse
import pandas as pd
from create_PSM_df import PSM_FDR
from ReadAccTaxon import ReadAccTaxon
from collections import defaultdict
from handling_acc_files import HelperMethod


In [10]:
def flatten_list(l):
    return [item for sublist in l for item in sublist]

In [11]:
def get_accs_from_df(x_tandem_result_tsv, db_type, decoy_tag):
    if db_type == 'uniprot' or db_type == 'swissprot':
        accs = set()
        for acc in pd.read_csv(str(x_tandem_result_tsv), delimiter='\t')['Protein'].tolist():
            try:
                accs.add(acc.split()[0].split('|')[1])
            # e.g. CRAP: KKA1_ECOLX
            except IndexError:
                accs.add(acc.split()[0])
    # ncbi: 'generic|AC:5988671|WP_080217951.1', 'generic|AC:2500558_REVERSED|EQV03804.1 ERA83742.1 WP_021536075.1-REVERSED'
    # problem: generic|AC:373747|pir||D85980 WP_000133047.1 AAG58304.1 -> maxsplit=2
    # CRAP accs starts with 'sp|' or Index Error
    elif db_type == 'ncbi':
        accs = set()
        for acc in pd.read_csv(str(x_tandem_result_tsv), delimiter='\t')['Protein'].tolist():
            try:
                if decoy_tag not in acc and not acc.startswith('sp|'):
                    accs.add(acc.split()[0].split('|', maxsplit=2)[2])
            except IndexError:
                accs.add(acc.split()[0])
    elif db_type == 'custom':
        accs = {acc.strip() for acc in pd.read_csv(str(x_tandem_result_tsv), delimiter='\t')['Protein'].tolist()}
    return accs

In [12]:
def get_ncbi_multiacc_to_accs_dict(ncbi_accs_from_file):
    path_to_multiaccs = '/home/jules/Documents/databases/databases_tax2proteome/multispecies_acc'
    # key = first acc in result tsv, value: all accs
    multacc_reader = ReadAccTaxon(db_path, db_type, path_to_multiaccs)
    multiacc2acc_dict = multacc_reader.read_multispecies_accs(ncbi_accs_from_file)
    return multiacc2acc_dict

In [13]:
def get_taxa_from_acc2taxid_files(db_path, db_type, all_accs, taxa):
    acc2tax_reader = ReadAccTaxon(db_path, db_type)
    acc_2_taxon_dict = acc2tax_reader.read_acc2tax(all_accs, taxa)
    return acc_2_taxon_dict

In [14]:
def create_acc_from_file_2_taxa_set_dict(multiacc2acc_dict, acc_2_taxon_dict, ncbi_accs_from_file):
    acc_in_tsv_2_taxa_set_dict = defaultdict(set)
    for acc in ncbi_accs_from_file:
        if acc in multiacc2acc_dict.keys():
            for m_acc in multiacc2acc_dict[acc]:
                try:
                    acc_in_tsv_2_taxa_set_dict[acc].add(int(acc_2_taxon_dict[m_acc]))
                    del acc_2_taxon_dict[m_acc]
                # acc not in acc_2_taxon_dict, because taxon was not in taxa_all_level
                except KeyError:
                    continue
        else:
            acc_in_tsv_2_taxa_set_dict[acc].add(int(acc_2_taxon_dict[acc]))
            del acc_2_taxon_dict[acc]
    return acc_in_tsv_2_taxa_set_dict
    

In [15]:
class PSM_FDR:
    def __init__(self, path_to_file, path_to_crap, decoy_tag):
        """
        :param path_to_file: path to xtandem output .xml converted to tsv
        one line one header with all accessions seperated by \t
        """
        self.path_to_file = path_to_file
        print('new version 2')
        self.crap_acc_set = self.get_all_crap_accs(path_to_crap)
        self.decoy_tag = decoy_tag
        self.decoy_list = ['DECOY', 'CRAP', 'DECOY/CRAP', 0, '0']
        self.sorted_xtandem_df = None
        self.fdr_pos = 0
        self.number_psms = 0
        self.decoys = 0

    def flatten_set(self, s):
        return {item for sublist in s for item in sublist}

    @staticmethod
    def get_all_crap_accs(path_to_crap):
        crap_acc_set = set()
        with open(str(path_to_crap)) as crap:
            for line in crap.readlines():
                if line.startswith('>'):
                    crap_acc_set.add(line[1:].strip())
        return crap_acc_set

    def add_level_specific_taxid_column(self, taxon_graph, level):
        self.sorted_xtandem_df[f'taxID_{level}'] = self.sorted_xtandem_df[f'taxID'].apply(lambda taxid:
                                                taxon_graph.find_level_up(taxid, level)
                                                if taxid not in self.decoy_list else taxid)
        return self.sorted_xtandem_df

    def group_df(self, level):
        reduced_df = self.sorted_xtandem_df.groupby(["#SpecFile", 'Title', 'Peptide', 'Hyperscore'], as_index=False).agg(
            {'Protein': lambda acc: set(acc), 'EValue': lambda x: set(list(x)), 'decoy': lambda x: set(x),
             'taxID': lambda taxid: set(taxid), f'taxID_{level}': lambda x: set(x)})
        return reduced_df

    def get_taxid_of_prot_acc(self, protein_acc, acc2tax_dict, db_type):
        if db_type=='ncbi':
            if protein_acc in self.crap_acc_set:
                return {'CRAP'}
            if self.decoy_tag in protein_acc:
                return {'DECOY'}
            try:
                # get taxa set
                return acc2tax_dict[protein_acc]
            except KeyError:
                print(protein_acc)
                return {'DECOY/CRAP'}
        else:
            if protein_acc in self.crap_acc_set:
                return 'CRAP'
            if self.decoy_tag in protein_acc:
                return 'DECOY'
            try:
                if db_type == 'uniprot' or db_type == 'swissprot':
                    return int(acc2tax_dict[protein_acc.split('|')[1]])
                elif db_type == 'custom':
                    return int(acc2tax_dict[protein_acc.strip()])
            except KeyError:
                print(protein_acc)
                return 'DECOY/CRAP'

    def create_PSM_dataframe_for_uniprot_accs(self, acc2tax_dict, taxon_graph, level):
        self.sorted_xtandem_df['Protein'] = self.sorted_xtandem_df['Protein'].apply(lambda protein_acc: protein_acc.split()[0])
        self.sorted_xtandem_df['taxID'] = self.sorted_xtandem_df['Protein'].apply(lambda protein_acc:
                                                                self.get_taxid_of_prot_acc(protein_acc, acc2tax_dict, 'uniprot'))

        self.sorted_xtandem_df = self.add_level_specific_taxid_column(taxon_graph, level)
        reduced_df = self.group_df(level)
        return reduced_df

    def get_first_acc(self, acc):
        if not acc.startswith('generic'):
            return acc
        decoy = True if self.decoy_tag in acc else False
        if decoy:
            return acc.split('|', maxsplit=2)[2].split()[0] + '_' + self.decoy_tag
        else:
            return acc.split('|', maxsplit=2)[2].split()[0]

    def create_PSM_dataframe_for_ncbi_accs(self, acc2tax_dict, taxon_graph, level):
        """
        :param acc2tax_dict: for ncbi multiple accs per accs posiible, so multiple taxa posiible: {acc string: {set of taxa(int)}
        :param taxon_graph: loaded TaxonGraph
        :param level: 'species', 'genus', ...
        :return: reduced_df same spectra ID and Hyperscore entries combined into one, all values into sets
        """
        # Multiaccs in Protein column: 'PNW76085.1 BAB64417.1 BAB64413.1 XP_001693987.1'
        print('get first accs in protein column')
        self.sorted_xtandem_df['Protein'] = \
            self.sorted_xtandem_df['Protein'].apply(lambda acc: self.get_first_acc(acc))
        print('get taxIDs of protein accs')
        self.sorted_xtandem_df['taxID'] = self.sorted_xtandem_df['Protein'].apply(lambda protein_acc:
                                                                                  self.get_taxid_of_prot_acc(protein_acc, acc2tax_dict, 'ncbi'))
        print(self.sorted_xtandem_df['taxID'][0:5])
        # 'Protein': generic|AC:4260012|KKY45073.1 WP_046834534.1, generic|AC:6172351|WP_086660554.1 ...
        # NCBI: 'taxID': {83334}, {562}, takes lot of time
        print('get taxIDs of specified level')
        self.sorted_xtandem_df[f'taxID_{level}'] = \
            self.sorted_xtandem_df['taxID'].apply(lambda taxID_set: {taxon_graph.find_level_up(int(taxID), level)
                                                                     if taxID not in self.decoy_list else taxID
                                                                     for taxID in taxID_set})
        print('reducing df')
        reduced_df = self.sorted_xtandem_df.groupby(["#SpecFile", 'Title', 'Peptide', 'Hyperscore'], as_index=False).agg(
            {'Protein': lambda acc: set(acc), 'EValue': lambda x: set(list(x)), 'decoy': lambda decoy: set(decoy),
             'taxID': lambda taxid_sets: self.flatten_set(taxid_sets),
             f'taxID_{level}': lambda taxid_sets: self.flatten_set(taxid_sets)})
        return reduced_df

    def create_PSM_dataframe_for_custom_accs(self, acc2tax_dict, taxon_graph, level):
        self.sorted_xtandem_df['taxID'] = self.sorted_xtandem_df['Protein'].apply(
            lambda acc: self.get_taxid_of_prot_acc(acc, acc2tax_dict))
        self.sorted_xtandem_df = self.add_level_specific_taxid_column(taxon_graph, level)
        reduced_df = self.group_df(level)
        return reduced_df

    def create_PSM_dataframe(self, db_type, level, taxon_graph, acc2tax_dict):
        """

        :return: Hyperscore sorted dataframe, Protein with highest Hyperscore first, Title = spectra_file_spectraID
        """
        xtandem_df = pd.read_csv(str(self.path_to_file), delimiter='\t')
        xtandem_df['Protein'] = xtandem_df['Protein'].apply(lambda acc: acc.strip())
        # change spectra Title
        xtandem_df['Title'] = xtandem_df['Title'].apply(lambda row: row.split(' File')[0])
        print('Sorting panda dataframe by hyperscore. Find taxa, and level taxa')
        self.sorted_xtandem_df = xtandem_df.sort_values(by=['Hyperscore', 'Title'], ascending=False).reset_index(drop=True)
        self.sorted_xtandem_df['decoy'] = self.sorted_xtandem_df.apply(lambda row: True if self.decoy_tag in row['Protein'] else False, axis=1)
        print('create dataframe with level taxa')
        if db_type == 'uniprot' or db_type == 'swissprot':
            reduced_df = self.create_PSM_dataframe_for_uniprot_accs(acc2tax_dict, taxon_graph, level)
        elif db_type == 'ncbi':
            reduced_df = self.create_PSM_dataframe_for_ncbi_accs(acc2tax_dict, taxon_graph, level)
        elif db_type == 'custom':
            reduced_df = self.create_PSM_dataframe_for_custom_accs(acc2tax_dict, taxon_graph, level)

        # sort by Hyperscore
        reduced_df = reduced_df.sort_values(by=['Hyperscore', 'Title'], ascending=False).reset_index(drop=True)

        print(f'entries in sorted df: {len(self.sorted_xtandem_df)} '
              f'decoys in sorted df: {len(self.sorted_xtandem_df[self.sorted_xtandem_df.decoy==True])} '
              f'hits in sorted df: {len(self.sorted_xtandem_df[self.sorted_xtandem_df.decoy==False])}')
        print(f'entries in reduced_df: {len(reduced_df)} '
              f'decoys in reduced_df: {len(reduced_df[reduced_df.decoy=={True}])} '
              f'hits in reduced_df: {len(reduced_df[reduced_df.decoy=={False}])} '
              f'mixed in reduced_df: {len(reduced_df[reduced_df.decoy=={False, True}])}')
        return reduced_df

In [16]:
def get_ncbi_acc2taxon_dict(ncbi_accs_from_file, db_path, db_type, taxa):
    """
    :param x_tandem_result_tsv:
    :param db_path:
    :param db_type:
    :param taxa: list of all relevant taxa, all taxa in taxon graph below specified level
    :return: acc_in_tsv_2_taxa_set_dict dict {acc string: {set of taxa(int)}
    """
    # Protein: generic|AC:2905764_REVERSED|VVN60222.1-REVERSED, generic|AC:4682148|PMZ83684.1 PMZ91489.1 WP_054593944.1 PMZ34596.1 ALI00421.1 PMZ37242.1...
    # generic|AC:3582431_REVERSED|WP_032249104.1 KDU12656.1 KDT75424.1-REVERSED
    
    
    # for ncbi find all multispecies accessions with file ncbi acc
    multiacc2acc_dict = get_ncbi_multiacc_to_accs_dict(ncbi_accs_from_file)
    # key = Multiacc or accs assigned to multiaccs
    all_multiaccs = set(flatten_list(multiacc2acc_dict.values()))
    all_accs = all_multiaccs.union(ncbi_accs_from_file)   
    acc_2_taxon_dict = get_taxa_from_acc2taxid_files(db_path, db_type, all_accs, taxa)
    acc_in_tsv_2_taxa_set_dict = create_acc_from_file_2_taxa_set_dict(all_accs, multiacc2acc_dict, acc_2_taxon_dict)
    
    return acc_in_tsv_2_taxa_set_dict

In [17]:
def get_rows_with_decoy_or_nan_in_reference_and_result(Ref_taxID_level_column, taxID_level_column):
        true_false_list = []
        for ref_taxID_set, taxID_set in zip(Ref_taxID_level_column, taxID_level_column):
            if ref_taxID_set == {'DECOY'} and taxID_set == {'DECOY'}:
                true_false_list.append(False)
            elif (ref_taxID_set == {'DECOY'} and type(taxID_set) != set) or (type(ref_taxID_set) != set and taxID_set == {'DECOY'}):
                true_false_list.append(False)
            else:
                true_false_list.append(True)
        return true_false_list
    
def get_all_spectra_IDs(ident_file):
    all_spec_IDs = set()
    with open(ident_file, 'r') as ident_file:
        for line in ident_file:
            if line.startswith('TITLE'):
                all_spec_IDs.add(line.split()[0].split('TITLE=')[1])
    return all_spec_IDs

def rename_reference_df_columns(reference_df):
    reference_df = reference_df[['Title', 'Peptide', 'Hyperscore', 'Protein', 'decoy', 'taxID', 'taxID_species']]
    reference_df.columns = ['Title', 'Ref_Peptide', 'Ref_Hyperscore', 'Ref_ProteinAcc', 'Ref_decoy', 'Ref_taxID_DB', 'Ref_taxID_species']
    return reference_df

def get_nan_or_decoy(ref_taxID_column, taxID_column):
    true_false_list = []
    for ref_taxID_set, taxID_set in zip(ref_taxID_column, taxID_column):
        if type(ref_taxID_set) != set and type(taxID_set) != set:
            true_false_list.append(True)
        elif (ref_taxID_set == {'DECOY'} and type(taxID_set) != set) or (type(ref_taxID_set) != set and taxID_set == {'DECOY'}):
            true_false_list.append(True)
        else:
            true_false_list.append(False)
    return true_false_list

def get_all_unidentified_spectra_in_reference_and_result(result_df, reduced_df):
    spectra_file = '/home/jules/Documents/Tax2Proteome/benchmarking/spectra/Run1_U1_2000ng.mgf'
    all_spectra_set = get_all_spectra_IDs(spectra_file)
    df_with_all_spectra_and_reference_and_results =  HelperMethod.create_df_with_all_spectra_and_dfs_to_compare(
            all_spectra_set, result_df, 'Title', reduced_df, 'Title')
    df_with_all_unidentified_spectra = df_with_all_spectra_and_reference_and_results[get_nan_or_decoy(df_with_all_spectra_and_reference_and_results.Ref_taxID_DB, df_with_all_spectra_and_reference_and_results.taxID)]
    return df_with_all_unidentified_spectra

def check_for_TP(taxid_set, taxid_ref_set):
    # TP: if all taxa from ref set contained in result set
    # ignore Decoy crap entries from result_reduced (not contained in reference)
    decoy_set = {'DECOY', 'DECOY/CRAP', 0}
    taxid_set = taxid_set.difference(decoy_set)
    if len(taxid_set) == 0 and pd.isna(taxid_ref_set): #only decoy entries
        return False
    return taxid_ref_set.issubset(taxid_ref_set)

def check_for_FP(taxid_set, taxid_ref_set):
    # FP: if all taxa from result_df not in reference_set
    if len(taxid_set.intersection(taxid_ref_set)) == 0:
        return True
    else:
        return False

def compare_tax_sets(taxid_set, taxid_ref_set, is_FP):
    if not pd.isna(taxid_set) and not pd.isna(taxid_ref_set):
        if is_FP:
            return check_for_FP(taxid_set, taxid_ref_set)
        else:
            return check_for_TP(taxid_set, taxid_ref_set)
    return False

def check_taxid_in_reference(taxid_level_column, taxid_level_ref_column, is_FP):
        true_false_list = []
        for taxid_set, taxid_ref_set in zip(taxid_level_column, taxid_level_ref_column):
            true_false_list.append(compare_tax_sets(taxid_set, taxid_ref_set, is_FP))
        return true_false_list

def get_true_positive_and_true_negative(df_with_all_unidentified_spectra, df_with_all_reference_spectra_and_merged_results):
        pd.set_option("display.max_rows", None, "display.max_columns", None)
        level = 'species'
        df_taxid = df_with_all_reference_spectra_and_merged_results[['Title','taxID','Ref_taxID_DB']]
        df_taxid_level = df_with_all_reference_spectra_and_merged_results[['Title', f'taxID_{level}',f'Ref_taxID_{level}']]
        # both nan or one nan one DECOY
        df_TN = df_with_all_unidentified_spectra
       # print(df_TN.head())
        TN=len(set(df_TN.SpectraID.tolist()))
        
        df_TP = df_taxid_level[check_taxid_in_reference(df_taxid_level[f'taxID_{level}'].tolist(), df_taxid_level[f'Ref_taxID_{level}'].tolist(), is_FP=False)]
        TP = len(set(df_TP.Title.tolist()))
        # FN: not identified in result, but identified in referernce
        df_FN = df_taxid[(df_taxid.taxID.isna() ) & df_taxid.Ref_taxID_DB.notna()]
        df_FN = df_FN[df_FN.Ref_taxID_DB != {'DECOY/CRAP'}]
        FN=len(set(df_FN.Title.tolist()))


        df_FP = df_taxid_level[check_taxid_in_reference(df_taxid_level[f'taxID_{level}'].tolist(), df_taxid_level[f'Ref_taxID_{level}'].tolist(), is_FP=True)]
        FP = len(set(df_FP.Title.tolist()))
        df_s = df_with_all_reference_spectra_and_merged_results[df_with_all_reference_spectra_and_merged_results.taxID != {'DECOY/CRAP'}]
        print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}, number of all spectra without decoy/crap spectra: "
              f"{len(set(df_s.Title))}, number of TP+TN+FP+FN: {TP+FN+FP+TN}")
        return TP, FP, TN, FN

def get_row_with_reference_nan_or_decoy_and_result_identified(ref_taxID_column, taxID_column):
    true_false_list = []
    for ref_taxID_set, taxID_set in zip(ref_taxID_column, taxID_column):
        if type(ref_taxID_set) != set and type(taxID_set) == set:
            if taxID_set not in [{'DECOY'},  {'DECOY/CRAP'}, {0}]:
                true_false_list.append(True)
            else:
                true_false_list.append(False)
        elif ref_taxID_set in [{'DECOY'},  {'DECOY/CRAP'}, {0}] and taxID_set not in [{'DECOY'},  {'DECOY/CRAP'}, {0}]:
            true_false_list.append(True)
        else:
            true_false_list.append(False)
    return true_false_list

In [27]:
from pathlib import Path
import argparse
import pandas as pd
from create_PSM_df import PSM_FDR
from ReadAccTaxon import ReadAccTaxon
from collections import defaultdict
from handling_acc_files import HelperMethod


def flatten_list(l):
    return [item for sublist in l for item in sublist]


def read_crap(file):
    crap = set()
    with open(file, "r") as f:
        for line in f:
            if line.startswith('>'):
                if '|' in line:
                    fields = [item for item in line.split('|')]
                    crap.add(fields[1])
                else:
                    crap.add(line[1:].strip())
    return crap

def get_accs_from_df(x_tandem_result_tsv, db_type, decoy_tag):
    acc_from_tsv = pd.read_csv(str(x_tandem_result_tsv), delimiter='\t')['Protein'].tolist()
    accs = get_accs_from_accs(acc_from_tsv, db_type, decoy_tag)
    return accs

def get_accs_from_accs(accs, db_type, decoy_tag):
    processed_accs = set()
    for acc in accs:
        if db_type == 'uniprot' or db_type == 'swissprot':
            try:
                if decoy_tag not in acc:
                    processed_accs.add(acc.split()[0].split('|')[1])
            # e.g. CRAP: KKA1_ECOLX
            except IndexError:
                processed_accs.add(acc.split()[0])
        elif db_type == 'ncbi':
            # ncbi: 'generic|AC:5988671|WP_080217951.1', 'generic|AC:2500558_REVERSED|EQV03804.1 ERA83742.1 WP_021536075.1-REVERSED'
            # problem: generic|AC:373747|pir||D85980 WP_000133047.1 AAG58304.1 -> maxsplit=2
            # CRAP accs starts with 'sp|' or Index Error
            try:
                # CRAP accs = sp|acc
                if decoy_tag not in acc and not acc.startswith('sp|'):
                    processed_accs.add(acc.split()[0].split('|', maxsplit=2)[2])
            except IndexError:
                accs.add(acc.split()[0])
        elif db_type == 'custom':
            if decoy_tag not in acc:
                processed_accs.add(acc.strip())
    return processed_accs


def get_acc2taxon_dict(db_path, db_type, accs=None):
    acc2tax_reader=ReadAccTaxon(db_path, db_type)
    if db_type == 'custom':
        acc_2_taxon_dict = acc2tax_reader.get_acc2taxonID_dict(db_path/'acc2tax_custom')
    elif db_type == 'uniprot' or db_type == 'swissprot':
        acc_2_taxon_dict = acc2tax_reader.read_acc2tax(accs)
    return acc_2_taxon_dict


def get_multispecies_accs(path_to_multispecies_acc, accs):
    """
    :param path_to_multispecies_acc: path to multispecies file
    :param accs: set of ncbi accs
    :return: multi_acc to list of accs dict
        """
    multiacc2accs_dict={}
    with open(path_to_multispecies_acc, 'r') as input:
        for line in input:
            fields = line.split()
            l = [acc in accs for acc in fields]
            if any(l):
                pos_of_acc = [i for i, x in enumerate(l) if x][0]
                multiacc2accs_dict[fields[pos_of_acc]] = fields[0:]
    return multiacc2accs_dict


def get_tsv_multspecies_acc_to_taxa_dict(multiacc2acc_dict, acc_2_taxon_dict):
    tsv_multi_acc_to_taxon_dict = defaultdict(set)
    for acc, acc_list in multiacc2acc_dict.items():
        for multi_acc in acc_list:
            if multi_acc in acc_2_taxon_dict.keys():
                tsv_multi_acc_to_taxon_dict[multi_acc].add(acc_2_taxon_dict[multi_acc])
    return tsv_multi_acc_to_taxon_dict


def remove_accs_with_unsupported_taxa(multi_acc_2_taxon_dict, taxa):
    accs_with_not_matching_taxa = []
    for acc, taxon in multi_acc_2_taxon_dict.items():
        if int(taxon) not in taxa:
            accs_with_not_matching_taxa.append(acc)
    for acc in accs_with_not_matching_taxa:
        del multi_acc_2_taxon_dict[acc]
    return multi_acc_2_taxon_dict


def get_ncbi_multiacc_to_accs_dict(ncbi_accs_from_file, db_path, db_type):
    path_to_multiaccs = '/home/jules/Documents/databases/databases_tax2proteome/multispecies_acc'
    # key = first acc in result tsv, value: all accs
    multacc_reader = ReadAccTaxon(db_path, db_type, path_to_multiaccs)
    multiacc2acc_dict = multacc_reader.read_multispecies_accs(ncbi_accs_from_file)
    return multiacc2acc_dict


def get_taxa_from_acc2taxid_files(db_path, db_type, all_accs, taxa):
    acc2tax_reader = ReadAccTaxon(db_path, db_type)
    acc_2_taxon_dict = acc2tax_reader.read_acc2tax(all_accs, taxa)
    return acc_2_taxon_dict


def create_acc_from_file_2_taxa_set_dict(ncbi_accs_from_file, multiacc2acc_dict, acc_2_taxon_dict):
    acc_in_tsv_2_taxa_set_dict = defaultdict(set)
    for acc in ncbi_accs_from_file:
        if acc in multiacc2acc_dict.keys():
            for m_acc in multiacc2acc_dict[acc]:
                try:
                    acc_in_tsv_2_taxa_set_dict[acc].add(int(acc_2_taxon_dict[m_acc]))
                    #del acc_2_taxon_dict[m_acc]
                # acc not in acc_2_taxon_dict, because taxon was not in taxa_all_level
                except KeyError:
                    continue
        else:
            acc_in_tsv_2_taxa_set_dict[acc].add(int(acc_2_taxon_dict[acc]))
            #del acc_2_taxon_dict[acc]
    return acc_in_tsv_2_taxa_set_dict


def get_ncbi_acc2taxon_dict(ncbi_accs_from_file, db_path, db_type, taxa):
    """
    :param x_tandem_result_tsv:
    :param db_path:
    :param db_type:
    :param taxa: list of all relevant taxa, all taxa in taxon graph below specified level
    :return: acc_in_tsv_2_taxa_set_dict dict {acc string: {set of taxa(int)}
    """
    # for ncbi find all multispecies accessions with file ncbi acc
    # key = Multiacc or accs assigned to multiaccs
    multiacc2acc_dict = get_ncbi_multiacc_to_accs_dict(ncbi_accs_from_file, db_path, db_type)
    all_accs = set(flatten_list(multiacc2acc_dict.values())).union(ncbi_accs_from_file)
    # acc_2_taxon_dict contains 0 values for some accessions (are in prot.acc...)
    acc_2_taxon_dict = get_taxa_from_acc2taxid_files(db_path, db_type, all_accs, taxa)
    all_accs.clear()
    # acc_in_tsv_2_taxa_set_dict contains in taxa_set items like '345' and '0'
    acc_in_tsv_2_taxa_set_dict = create_acc_from_file_2_taxa_set_dict(ncbi_accs_from_file, multiacc2acc_dict, acc_2_taxon_dict)
    multiacc2acc_dict.clear()
    return acc_in_tsv_2_taxa_set_dict

def create_reduced_df(db_path, path_to_x_tandem_result_tsv, db_type, level, taxonset, add_db=None, taxon=None):
    Kleiner_taxIDs = [262724, 882, 176299, 228410, 44577, 926571, 323848, 12022, 1283336, 10754, 101570, 224308, 1041145,
                      1004788, 266265, 266264, 99287, 1302247, 1149133, 3055, 93061, 1977402, 1114970, 511145,
                      536, 1407502, 1294143,1619948]
    Tanca_taxIDs = [747, 5535, 655183, 1579, 1255, 4932, 1465, 1351, 562]
    if taxonset == 'kleiner':
        taxonIDs = Kleiner_taxIDs
        levels = ['subspecies', 'species', 'genus', 'family', 'order', 'superkingdom']
    elif taxonset == 'tanca':
        taxonIDs = Tanca_taxIDs
        levels = ['species', 'genus', 'family', 'order', 'superkingdom']
    if taxon:
        taxonIDs.extend([taxID for taxonlist in options.taxon for taxID in taxonlist])

    path_to_x_tandem_result_tsv = Path(path_to_x_tandem_result_tsv)
    path_to_crap = Path(options.crap)
    decoy_tag = options.decoy
    taxon_graph = HelperMethod.load_taxa_graph(Path(options.tax_graph))

    # acc_2_taxon_dict for identification file identified accs
    if db_type == 'custom':
        accs_in_tsv = set()
    else:
        accs_in_tsv = get_accs_from_df(path_to_x_tandem_result_tsv, db_type, decoy_tag)


    if db_type == 'ncbi':
        all_taxa_of_level_set = set(flatten_list(taxon_graph.get_all_taxids(taxonIDs, level)))
        # acc_in_tsv_2_taxa_set_dict
        acc_2_taxon_dict = get_ncbi_acc2taxon_dict(accs_in_tsv, db_path, db_type, all_taxa_of_level_set)
    else:
        acc_2_taxon_dict = get_acc2taxon_dict(db_path, db_type, accs_in_tsv)
    accs_in_tsv.clear()

    if add_db:
        # accs= full accs without split, if custom: get dict of all accs in db
        accs = get_accs_from_df(path_to_x_tandem_result_tsv, 'custom', decoy_tag)
        add_accs = {acc for acc in accs if acc not in acc_2_taxon_dict.keys()}
        processed_add_acc = get_accs_from_accs(add_accs, options.add_db, decoy_tag)
        processed_add_acc_2_taxon_dict = get_acc2taxon_dict(db_path, options.add_db, processed_add_acc)
        add_acc_2_taxon_dict = {}
        for processed_acc, taxon in processed_add_acc_2_taxon_dict.items():
            for acc in add_accs:
                if processed_acc in acc:
                    add_acc_2_taxon_dict[acc] = taxon
        acc_2_taxon_dict.update(add_acc_2_taxon_dict)
    for k, v in acc_2_taxon_dict.items():
        if v == 0 or v=='0':
            print(k,v)

    psm = PSM_FDR(path_to_x_tandem_result_tsv, path_to_crap, decoy_tag)
    reduced_df = psm.create_PSM_dataframe(db_type, level, taxon_graph, acc_2_taxon_dict)
    print(f"writing data frame to {path_to_x_tandem_result_tsv.parent.joinpath(path_to_x_tandem_result_tsv.stem + '_reduced.tsv')}... ")
    reduced_df.to_csv(str(path_to_x_tandem_result_tsv.parent.joinpath(path_to_x_tandem_result_tsv.stem + '_reduced.tsv')), sep='\t')

In [35]:
# files and paths
path_to_crap ='/home/jules/Documents/Tax2Proteome/benchmarking/Kleiner_ref_db/crap.fasta'
db_path = Path('/home/jules/Documents/databases/databases_tax2proteome/')
input_kleiner_reference_aradiopsis = '/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml.tsv.tsv'
path_to_kleiner_acc2tax = '/home/jules/Documents/Tax2Proteome/benchmarking/Kleiner_ref_db'
crap_file = '/home/jules/Documents/Tax2Proteome/benchmarking/Kleiner_ref_db/crap.fasta'
path_to_taxdump = '/home/jules/Documents/databases/databases_tax2proteome/taxdump.tar.gz'
path_to_ncbi_species_x_tandem_result_tsv = Path('/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species.t.xml.tsv')

In [19]:
# taxa
Kleiner_taxIDs = [262724, 882, 176299, 228410, 44577, 926571, 323848, 12022, 1283336, 10754, 101570, 224308, 216596,
                      1004788, 266265, 266264, 99287, 1294143, 1149133, 3055, 1280, 1977402, 294, 83333,
                      536, 1407502]
Tanca_taxIDs = [747, 5535, 655183, 1579, 1255, 4932, 1465, 1351, 562]
decoy_tag = 'REVERSED'

In [ ]:
# NCBI
db_type = 'ncbi'
level = 'species'
%run create_reduced_df_with_taxa.py -i /home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml.tsv -p /home/jules/Documents/databases/databases_tax2proteome/ -c /home/jules/Documents/Tax2Proteome/benchmarking/Kleiner_ref_db/crap.fasta -g /home/jules/Documents/databases/databases_tax2proteome/taxdump.tar.gz -l subspecies -d ncbi -s kleiner

In [31]:
# NCBI step by step
path_to_x_tandem_result_tsv=Path("/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml.tsv")
accs = get_accs_from_df(path_to_x_tandem_result_tsv, "ncbi", decoy_tag)

In [36]:
taxon_graph = HelperMethod.load_taxa_graph(Path(path_to_taxdump))
all_taxa_of_level_set = set(flatten_list(taxon_graph.get_all_taxids(Kleiner_taxIDs, 'species')))

Load taxon graph from harddrive.


In [38]:
# acc_2_taxon_dict = get_ncbi_acc2taxon_dict(accs, db_path, db_type, all_taxa_of_level_set)
multiacc2acc_dict = get_ncbi_multiacc_to_accs_dict(accs, db_path, 'ncbi')

Start reading multiaccs database file with 8 threads.
10% read.
20% read.
30% read.
40% read.
50% read.
60% read.
70% read.
80% read.
90% read.


In [39]:
all_accs = set(flatten_list(multiacc2acc_dict.values())).union(accs)

In [41]:
# acc_2_taxon_dict contains 0 values for some accessions (are in prot.acc...) -> not without del in function
acc_2_taxon_dict = get_taxa_from_acc2taxid_files(db_path, db_type, all_accs, all_taxa_of_level_set)

Start reading accession2prot database file with 8 threads.
10% read.
20% read.
30% read.
40% read.
50% read.
60% read.
70% read.
80% read.
90% read.


In [43]:
#all_accs.clear()
# acc_in_tsv_2_taxa_set_dict contains in taxa_set items like '345' and '0'
acc_in_tsv_2_taxa_set_dict = create_acc_from_file_2_taxa_set_dict(accs, multiacc2acc_dict, acc_2_taxon_dict)
#multiacc2acc_dict.clear()
acc_2_taxon_dict = acc_in_tsv_2_taxa_set_dict

In [48]:
for k, v in acc_2_taxon_dict.items():
        if v == 0 or v=='0':
            print(k,v)


In [52]:
psm = PSM_FDR(path_to_x_tandem_result_tsv, path_to_crap, decoy_tag)
reduced_df = psm.create_PSM_dataframe(db_type, level, taxon_graph, acc_2_taxon_dict)

Sorting panda dataframe by hyperscore. Find taxa, and level taxa
create dataframe with level taxa
get first accs in protein column
get taxIDs of protein accs
get taxIDs of specified level
reducing df
entries in sorted df: 311814 decoys in sorted df: 93867 hits in sorted df: 217947
entries in reduced_df: 131639 decoys in reduced_df: 40319 hits in reduced_df: 91232 mixed in reduced_df: 88


In [53]:
print(f"writing data frame to {Path(path_to_x_tandem_result_tsv).parent.joinpath(Path(path_to_x_tandem_result_tsv).stem + '_reduced.tsv')}... ")
reduced_df.to_csv(str(Path(path_to_x_tandem_result_tsv).parent.joinpath(Path(path_to_x_tandem_result_tsv).stem + '_reduced.tsv')), sep='\t')

writing data frame to /home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_reduced.tsv... 


In [59]:
# Uniprot
%run create_reduced_df_with_taxa.py -i /home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies_nr.t.xml.tsv -p /home/jules/Documents/databases/databases_tax2proteome/ -c /home/jules/Documents/Tax2Proteome/benchmarking/Kleiner_ref_db/crap.fasta -g /home/jules/Documents/databases/databases_tax2proteome/taxdump.tar.gz -l species -d uniprot -s kleiner

Load taxon graph from harddrive.
Start reading accession2prot database file with 8 threads.
10% read.
20% read.
30% read.
40% read.
50% read.
60% read.
70% read.
80% read.
90% read.
Sorting panda dataframe by hyperscore. Find taxa, and level taxa
create dataframe with level taxa
entries in sorted df: 201790 decoys in sorted df: 61298 hits in sorted df: 140492
entries in reduced_df: 130684 decoys in reduced_df: 41153 hits in reduced_df: 89445 mixed in reduced_df: 86
writing data frame to /home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies_nr.t.xml_reduced.tsv... 


In [ ]:
reduced_df = psm.create_PSM_dataframe(db_type, level, taxon_graph, acc_2_taxon_dict)
print(f"writing data frame to {path_to_ncbi_species_x_tandem_result_tsv.parent.joinpath(path_to_x_tandem_result_tsv.stem + '_reduced.tsv')}... ")
reduced_df.to_csv(str(path_to_ncbi_species_x_tandem_result_tsv.parent.joinpath(path_to_ncbi_species_x_tandem_result_tsv.stem + '_reduced.tsv')), sep='\t')

In [ ]:
%run anaylze_identification_files.py -p /home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies_nr.t.xml.tsv

In [ ]:
 # NCBI
db_path = Path('/home/jules/Documents/databases/databases_tax2proteome/')
db_type = 'ncbi'
# for reduced_df True
is_decoy_column_set = True
x_tandem_result_tsv = '/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species.t.xml.tsv'
decoy_tag = 'REVERSED'

In [ ]:
all_taxa_of_level_set = set(flatten_list(taxon_graph.get_all_taxids(taxonIDs, 'species')))

In [ ]:
acc_2_taxon_dict = get_ncbi_acc2taxon_dict(x_tandem_result_tsv, db_path, db_type, all_taxa_of_level_set, decoy_tag)

In [ ]:
list(acc_2_taxon_dict.items())[0:5]

In [ ]:
psm = PSM_FDR(x_tandem_result_tsv)
reduced_df = psm.create_PSM_dataframe(decoy_tag, db_type, 'species', taxon_graph, acc2tax_dict=acc_2_taxon_dict)

In [ ]:
reduced_df.head()

In [ ]:
fdr_pos, number_psms, decoys = psm.determine_FDR_position(reduced_df, options.fdr, is_decoy_column_set)

In [ ]:
x_tandem_result_tsv = '/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus.t.xml.tsv'
all_taxa_of_level_set = set(flatten_list(taxon_graph.get_all_taxids(taxonIDs, 'genus')))
acc_2_taxon_dict = get_ncbi_acc2taxon_dict(x_tandem_result_tsv, db_path, db_type, all_taxa_of_level_set, decoy_tag)

In [ ]:
psm = PSM_FDR(x_tandem_result_tsv)
reduced_df_genus = psm.create_PSM_dataframe(decoy_tag, db_type, 'species', taxon_graph, acc2tax_dict=acc_2_taxon_dict)
reduced_df_genus.head()

In [ ]:
reduced_df = ReferenceWriter.read_csv_with_generic_function('/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species.t.xml_reduced.tsv',
                                            ['Protein', 'decoy', 'taxID', f'taxID_species'])


In [ ]:
psm = PSM_FDR(x_tandem_result_tsv)
fdr_pos, number_psms, decoys = psm.determine_FDR_position(reduced_df, 0.05, is_decoy_column_set)

In [ ]:
reference_df = ReferenceWriter.read_csv_with_generic_function('/home/jules/Documents/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv',
                                           ['Protein', 'decoy', 'taxID', f'taxID_species'])
reference_df = reference_df[['Title', 'Peptide', 'Hyperscore', 'Protein', 'decoy', 'taxID', 'taxID_species']]
fdr_pos1, number_psms1, decoys1 = psm.determine_FDR_position(reduced_df, 0.05, True)
fdr_pos2, number_psms2, decoys2 = psm.determine_FDR_position(reference_df, 0.05, True)
fdr_applied_reference_df = reference_df[0:fdr_pos2]
fdr_applied_reduced_df =reduced_df[0:fdr_pos1]
fdr_applied_reference_df.columns = ['Title', 'Ref_Peptide', 'Ref_Hyperscore', 'Ref_ProteinAcc', 'Ref_decoy', 'Ref_taxID_DB', 'Ref_taxID_species']
print(fdr_applied_reference_df.head())

In [ ]:
df_with_all_reference_spectra_and_merged_results = pd.merge(fdr_applied_reference_df, fdr_applied_reduced_df, how="left",
                                                                    left_on='Title', right_on='Title')

In [ ]:
# remove DECOYs
df_with_all_reference_spectra_and_merged_results = df_with_all_reference_spectra_and_merged_results[get_rows_with_decoy_or_nan_in_reference_and_result(df_with_all_reference_spectra_and_merged_results.Ref_taxID_species, df_with_all_reference_spectra_and_merged_results.taxID_species)]
df_with_all_reference_spectra_and_merged_results.tail(5)

In [ ]:
#print(reference_df.head())
# reference_df = rename_reference_df_columns(reference_df)
df_with_all_unidentified_spectra = get_all_unidentified_spectra_in_reference_and_result(reference_df, reduced_df)
#print(df_with_all_unidentified_spectra.tail())

In [ ]:
get_true_positive_and_true_negative(df_with_all_unidentified_spectra, df_with_all_reference_spectra_and_merged_results)

In [ ]:
df_with_all_result_spectra_and_merged_reference_in_fdr = pd.merge(fdr_applied_reduced_df, fdr_applied_reference_df, how="left",
                                                                           on='Title')
df_only_result = df_with_all_result_spectra_and_merged_reference_in_fdr[get_row_with_reference_nan_or_decoy(
df_with_all_result_spectra_and_merged_reference_in_fdr.Ref_taxID_DB, df_with_all_result_spectra_and_merged_reference_in_fdr.taxID)]
df_only_result.head(3)
print(len(df_only_result))
len(set(df_only_result.Title.tolist()))
